In [1]:
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn import metrics

In [2]:
def getCertainDayCertainBehavior(user_behavior_data_path,start_day,end_day,bt):
    second_end_day = (end_day - datetime.timedelta(days=1))    
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    print("end day:",end_day_str)
    second_end_day_str = str(second_end_day.date())
    #output file name
    part_data_file_name = "../DataSet/Rules/"+"part_data"+end_day_str+"b"+str(bt)+".csv"
    
    item_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_item.csv')
    #predict_data_set_buy = predict_data_set_buy[predict_data_set_buy.item_id.isin(list(item_table.item_id))]
    #predict_data_set_buy[['user_id','item_id']].to_csv("../DataSet/tianchi_mobile_recommendation_predict.csv",index=None)
    print("read sub set end")
        
    batch = 0
    dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d %H')
    for df in pd.read_csv(open(user_behavior_data_path, 'r'), 
                      parse_dates=['time'], 
                      index_col = ['time'], 
                      date_parser = dateparse,
                      chunksize = 10000000):  # operation on chunk as the data file is too large
        try:
            part_data = df[end_day_str]
            part_data = part_data[part_data.behavior_type==bt]
            part_data = part_data[part_data.item_id.isin(list(item_table.item_id))] #included in sub set
            part_data.to_csv(part_data_file_name,  
                             columns=['user_id','item_id','behavior_type'],
                             header=False, mode='a')              
            
            batch += 1
            print('chunk %d done.' %batch) 
        except StopIteration:
            print("divide the data set finish.")
            break 
    print("end")
    pass

In [3]:
'''test = pd.read_csv('../DataSet/DevideData/part_data2014-12-09to2014-12-14.csv')
test.columns = ['time','user_id','item_id','behavior_type','test']
test[test.time>'2014-12-14']'''

"test = pd.read_csv('../DataSet/DevideData/part_data2014-12-09to2014-12-14.csv')\ntest.columns = ['time','user_id','item_id','behavior_type','test']\ntest[test.time>'2014-12-14']"

In [4]:
def doNotBuyUser(user_behavior_data_path,start_day,end_day):
    second_end_day = (end_day - datetime.timedelta(days=1))    
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    print("end day:",end_day_str)
    second_end_day_str = str(second_end_day.date())
    print("second_end_day:",second_end_day)
    name = start_day_str + "to" + second_end_day_str
    #output file name
    part_data_file_name = "../DataSet/Rules/"+"notbuyusers"+name+".csv"
    
    item_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_item.csv')
    #predict_data_set_buy = predict_data_set_buy[predict_data_set_buy.item_id.isin(list(item_table.item_id))]
    #predict_data_set_buy[['user_id','item_id']].to_csv("../DataSet/tianchi_mobile_recommendation_predict.csv",index=None)
    print("read sub set end")
        
    batch = 0
    dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d %H')
    for df in pd.read_csv(open(user_behavior_data_path, 'r'), 
                      parse_dates=['time'], 
                      index_col = ['time'], 
                      date_parser = dateparse,
                      chunksize = 10000000):  # operation on chunk as the data file is too large
        try:
            part_data = df[start_day_str:second_end_day_str]  #has problem
            part_data_buy = part_data[part_data.behavior_type==4]
            part_data_buy = part_data_buy.drop_duplicates(['user_id'])
'''            part_data_3 = part_data[part_data.behavior_type==3]
            part_data_2 = part_data[part_data.behavior_type==2]
            part_data_2 = part_data[part_data.behavior_type==2]'''
            part_data = part_data[~part_data.user_id.isin(list(part_data_buy.user_id))] #included in sub set
            part_data = part_data.drop_duplicates(['user_id'])
            part_data.to_csv(part_data_file_name,  
                             columns=['user_id','item_id','behavior_type'],
                             header=False, mode='a')              
            
            batch += 1
            print('chunk %d done.' %batch) 
        except StopIteration:
            print("divide the data set finish.")
            break 
    print("end")
    pass

In [21]:
'''start = datetime.datetime(2014,12,3,0,0,0)
end = datetime.datetime(2014,12,17,0,0,0)
doNotBuyUser("../DataSet/tianchi_fresh_comp_train_user.csv", start, end)'''

end day: 2014-12-17
second_end_day: 2014-12-16 00:00:00
read sub set end
chunk 1 done.
chunk 2 done.
chunk 3 done.
end


In [9]:
#rule1
part_data = pd.read_csv("../DataSet/Rules/part_data2014-12-17b3.csv", header=None, index_col=False)
part_data.columns = ['time','user_id','item_id','behavior_type']
part_data.drop_duplicates(['user_id','item_id'])
part_data['label'] = 1
#rule2
part_data_test = pd.read_csv("../DataSet/Rules/part_data2014-12-18b4.csv", header=None, index_col=False)
part_data_test.columns = ['time','user_id','item_id','behavior_type']
print('shape:',part_data_test.shape)
part_data_test.drop_duplicates(['user_id','item_id'])
part_data_test['label']=1
part_data_test.head(5)

part_data_b4 = pd.read_csv("../DataSet/Rules/part_data2014-12-17b4.csv", header=None, index_col=False)
part_data_b4.columns = ['time','user_id','item_id','behavior_type']
remove_from_b3 = part_data.merge(part_data_b4,on=['item_id','user_id'],how='left').fillna(-1)
part_data = remove_from_b3[remove_from_b3.behavior_type_y==-1]
#rule3:先买了，在加购物车的。事实证明先买了的再加购物车的第二天也不会买
'''a =remove_from_b3[remove_from_b3.time_y !=-1]
b = a[a.time_y < a.time_x]
part_data = pd.concat([part_data, b])'''
#rule4:当天有过购买行为的用户都去出掉
part_data = part_data[~part_data.user_id.isin(list(part_data_b4.user_id))] #included in sub set

#rule5:前半个月从买没买过东西的去处掉
'''not_buy_users = pd.read_csv("../DataSet/Rules/notbuyusers2014-12-03to2014-12-16.csv", header=None, index_col=False)
not_buy_users.columns = ['time','user_id','item_id','behavior_type']
part_data = part_data[~part_data.user_id.isin(list(not_buy_users.user_id))] #included in sub set

print('part_data:\n',part_data)'''

predict_data_set_buy = part_data
reference_data_set_buy = part_data_test
right_res = predict_data_set_buy.merge(reference_data_set_buy,on=['item_id','user_id'])

#caculate P, R and F1  
R = right_res.shape[0]/reference_data_set_buy.shape[0]
P = right_res.shape[0]/predict_data_set_buy.shape[0]
F1 = 2*P*R/(P+R)
print('reference_data_set_buy:',reference_data_set_buy.shape[0])
#print('predict_data_set:',predict_data_set.shape[0])
print('predict_data_set_buy:',predict_data_set_buy.shape[0])
print('right_res:',right_res.shape[0])
print('P:',P,'\n','R:',R,'\n','F1:',F1)


shape: (781, 4)
part_data:
                    time_x    user_id    item_id  behavior_type_x  label  \
2     2014-12-17 00:00:00  100442521  244784735                3      1   
3     2014-12-17 15:00:00  100442521  225667411                3      1   
4     2014-12-17 00:00:00  100442521  127658419                3      1   
5     2014-12-17 00:00:00  100442521   87030081                3      1   
6     2014-12-17 00:00:00  100442521  127819102                3      1   
7     2014-12-17 00:00:00  100442521  362002225                3      1   
8     2014-12-17 13:00:00  100442521  333445454                3      1   
9     2014-12-17 07:00:00  101105140   16585103                3      1   
14    2014-12-17 19:00:00  103424518  301444118                3      1   
15    2014-12-17 22:00:00  103490614  226466959                3      1   
16    2014-12-17 14:00:00  104428489  120341900                3      1   
18    2014-12-17 12:00:00  104982265  277789830                3      1 

In [ ]:
1、拿12月17日加购物车的数据预测为12月18日购买的user-item对。
reference_data_set_buy: 781
predict_data_set_buy: 1764
right_res: 61
P: 0.03458049886621315 
 R: 0.07810499359795134 
 F1: 0.04793713163064833
2、在1的基础上，将12月17日加入购物车的，同时也购买过的user-item对从预测数据集中去除
reference_data_set_buy: 781
predict_data_set_buy: 1415
right_res: 59
P: 0.04169611307420495 
 R: 0.07554417413572344 
 F1: 0.05373406193078325
3、在2的基础上，将买了之后又加购物车的也作为预测内容。但数据证明，买了之后再加购物车的第二天也不会买了。   
reference_data_set_buy: 781
predict_data_set_buy: 1427
right_res: 59
P: 0.04134548002803083 
 R: 0.07554417413572344 
 F1: 0.05344202898550724
4
[1056 rows x 7 columns]
reference_data_set_buy: 781
predict_data_set_buy: 1056
right_res: 52
P: 0.04924242424242424 
 R: 0.06658130601792574 
 F1: 0.056614044637996734
5
877 rows x 7 columns]
reference_data_set_buy: 781
predict_data_set_buy: 877
right_res: 39
P: 0.04446978335233751 
 R: 0.0499359795134443 
 F1: 0.047044632086851626

In [ ]:
def rule1(data):
    part_data = pd.read_csv("part_data2014-12-17b3.csv", header=None, index_col=False)
    part_data.columns = [user_id','item_id','behavior_type']
    part_data.drop_duplicates([])
    
    

In [3]:
start = datetime.datetime(2014,12,5,0,0,0)
end = datetime.datetime(2014,12,18,0,0,0)
getCertainDayCertainBehavior("../DataSet/tianchi_fresh_comp_train_user.csv", start, end,3)

read sub set end
chunk 1 done.
chunk 2 done.
chunk 3 done.


In [6]:
start = datetime.datetime(2014,12,5,0,0,0)
end = datetime.datetime(2014,12,18,0,0,0)
getCertainDayCertainBehavior("../DataSet/tianchi_fresh_comp_train_user.csv", start, end,4)

end day: 2014-12-18
read sub set end
chunk 1 done.
chunk 2 done.
chunk 3 done.
end


In [7]:
start = datetime.datetime(2014,12,5,0,0,0)
end = datetime.datetime(2014,12,17,0,0,0)
getCertainDayCertainBehavior("../DataSet/tianchi_fresh_comp_train_user.csv", start, end,3)

end day: 2014-12-17
read sub set end
chunk 1 done.
chunk 2 done.
chunk 3 done.
end


In [8]:
start = datetime.datetime(2014,12,5,0,0,0)
end = datetime.datetime(2014,12,17,0,0,0)
getCertainDayCertainBehavior("../DataSet/tianchi_fresh_comp_train_user.csv", start, end,4)

end day: 2014-12-17
read sub set end
chunk 1 done.
chunk 2 done.
chunk 3 done.
end


In [10]:
#最终提交答案的代码
#rule1
part_data = pd.read_csv("../DataSet/Rules/part_data2014-12-18b3.csv", header=None, index_col=False)
part_data.columns = ['time','user_id','item_id','behavior_type']
part_data.drop_duplicates(['user_id','item_id'])
part_data['label'] = 1

#rule2
part_data_b4 = pd.read_csv("../DataSet/Rules/part_data2014-12-18b4.csv", header=None, index_col=False)
part_data_b4.columns = ['time','user_id','item_id','behavior_type']
remove_from_b3 = part_data.merge(part_data_b4,on=['item_id','user_id'],how='left').fillna(-1)
part_data = remove_from_b3[remove_from_b3.behavior_type_y==-1]
#rule3:先买了，在加购物车的。事实证明先买了的再加购物车的第二天也不会买
'''a =remove_from_b3[remove_from_b3.time_y !=-1]
b = a[a.time_y < a.time_x]
part_data = pd.concat([part_data, b])'''
#rule4:当天有过购买行为的用户都去出掉
part_data = part_data[~part_data.user_id.isin(list(part_data_b4.user_id))] #included in sub set

#rule5:前半个月从买没买过东西的去处掉
'''not_buy_users = pd.read_csv("../DataSet/Rules/notbuyusers2014-12-03to2014-12-16.csv", header=None, index_col=False)
not_buy_users.columns = ['time','user_id','item_id','behavior_type']
part_data = part_data[~part_data.user_id.isin(list(not_buy_users.user_id))] #included in sub set

print('part_data:\n',part_data)'''
item_table = pd.read_csv('../DataSet/tianchi_fresh_comp_train_item.csv')
part_data = part_data[part_data.item_id.isin(list(item_table.item_id))] #included in sub set
part_data[['user_id','item_id']].to_csv("../DataSet/Rules/tianchi_mobile_recommendation_predict.csv",index=None)
